In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15166135
paper_name = 'lesage_bussey_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table3-4.xlsx', sheet_name='Table 3', skiprows=2)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 38 x 5


In [7]:
original_data.head()

,ORF,Gene,Comment,Haploid,Heterozygous diploid
0,Cell wall components synthesis and cell wall a...,NaN,NaN,NaN,NaN
1,YBR023C,CHS3,Chitin synthase III,--,--
2,YBL061C,CHS4,Activator of Chs3p,--,NaN
3,YLR330W,CHS5,Involved in Chs3p transport from the late Golg...,--,NaN
4,YJL099W,CHS6,Involved in Chs3p transport from the late Golg...,--,NaN


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

                                                           ORF Gene Comment  \
index_input                                                                   
0            Cell wall components synthesis and cell wall a...  NaN     NaN   
10               Cytoskeleton assembly and vesicular transport  NaN     NaN   
16                         Transcription and protein synthesis  NaN     NaN   
31                                 Other and unknown functions  NaN     NaN   

            Haploid Heterozygous diploid  \
index_input                                
0               NaN                  NaN   
10              NaN                  NaN   
16              NaN                  NaN   
31              NaN                  NaN   

                                                        orf  
index_input                                                  
0            CELLWALLCOMPONENTSSYNTHESISANDCELLWALLASSEMBLY  
10                CYTOSKELETONASSEMBLYANDVESICULARTRANSPORT  
16          

In [12]:
original_data = original_data.loc[t,:]

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
# Separate hap from het data
original_data1 = original_data.loc[original_data['Haploid'].isnull(),]
original_data2 = original_data.loc[original_data['Haploid'].notnull(),]

In [15]:
original_data1['data'] = original_data1['Heterozygous diploid'].apply(lambda x: -len(x.strip()))

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [16]:
original_data2['data'] = original_data2['Haploid'].apply(lambda x: -len(x.strip()))

/opt/anaconda3/envs/yp-data/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
original_data1 = original_data1[['data']].copy()
original_data2 = original_data2[['data']].copy()

In [18]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

# Load data (2)

In [19]:
original_data3 = pd.read_excel('raw_data/Table3-4.xlsx', sheet_name='Table 4')

In [20]:
original_data3.head()

,ORF,Gene,Comment
0,Cell wall assembly,NaN,NaN
1,YGR032W,FKS2,"β-1,3-Glucan synthase subunit"
2,YOR008C,SLG1,Sensor for the PKC1-SLT2 cell integrity pathway
3,YLR425W,TUS1,GDP-GTP exchange factor for Rho1p
4,Cytoskeleton and vesicular transport,NaN,NaN


In [21]:
original_data3['orf'] = original_data3['ORF'].astype(str)

In [22]:
# Eliminate all white spaces & capitalize
original_data3['orf'] = clean_orf(original_data3['orf'])

In [23]:
# Translate to ORFs 
original_data3['orf'] = translate_sc(original_data3['orf'], to='orf')

In [24]:
# Make sure everything translated ok
t = looks_like_orf(original_data3['orf'])
print(original_data3.loc[~t,])

                                              ORF Gene Comment  \
index_input                                                      
0                              Cell wall assembly  NaN     NaN   
4            Cytoskeleton and vesicular transport  NaN     NaN   
9                  Stress and signal transduction  NaN     NaN   
15            Transcription and protein synthesis  NaN     NaN   
18                    Other and unknown functions  NaN     NaN   

                                           orf  
index_input                                     
0                             CELLWALLASSEMBLY  
4            CYTOSKELETONANDVESICULARTRANSPORT  
9                  STRESSANDSIGNALTRANSDUCTION  
15            TRANSCRIPTIONANDPROTEINSYNTHESIS  
18                    OTHERANDUNKNOWNFUNCTIONS  


In [25]:
original_data3 = original_data3.loc[t,:]

In [26]:
original_data3.set_index('orf', inplace=True)

In [27]:
original_data3['data'] = 1

In [28]:
original_data3 = original_data3[['data']].copy()

In [29]:
original_data3 = original_data3.groupby(original_data3.index).mean()

# Merge

In [30]:
original_data = pd.concat([original_data2, original_data3], axis=0)

In [33]:
original_data = pd.concat([original_data, original_data1], axis=1)

In [37]:
original_data.shape

(61, 2)

In [44]:
original_data.index.name='orf'

# Prepare the final dataset

In [45]:
data = original_data.copy()

In [46]:
dataset_ids = [21846,21848]
datasets = datasets.reindex(index=dataset_ids)

In [47]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [48]:
data.head()

dataset_id,21846,21848
data_type,value,value
orf,,
YBL007C,-1.0,NaN
YBL061C,-2.0,NaN
YBR023C,-2.0,NaN
YDL047W,-1.0,NaN
YDR173C,-2.0,NaN


## Subset to the genes currently in SGD

In [49]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [50]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21846,21848
,data_type,value,value
gene_id,orf,,
95,YBL007C,-1.0,NaN
149,YBL061C,-2.0,NaN
219,YBR023C,-2.0,NaN
1925,YDL047W,-1.0,NaN
2299,YDR173C,-2.0,NaN


# Normalize

In [51]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [52]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [53]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,21846,21848,21846,21848
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
95,YBL007C,-1.0,NaN,-6.338921,NaN
149,YBL061C,-2.0,NaN,-12.742525,NaN
219,YBR023C,-2.0,NaN,-12.742525,NaN
1925,YDL047W,-1.0,NaN,-6.338921,NaN
2299,YDR173C,-2.0,NaN,-12.742525,NaN


# Print out

In [54]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [55]:
from IO.save_data_to_db3 import *

In [56]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 15166135...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00,  6.36it/s]


Updating the data_modified_on field...
